In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pandas_profiling

In [2]:
'''
Procedure: removeGranular
Inputs:
    df         Dataframe     
Outputs:
    Dataframe
Purpose:
Imported data from the Food Balance Sheet of the Food and Agriculture Organization of the United Nations
is filtered to exclude data not needed in this analysis. That is, granular details such as specific fruits, 
veggies, and dairy are removed. As are pre-categories country groupings. 
Unused columns are also dropped for reasons specified below
'''
def removeGranular(df):
    df = df.loc[(
                   (df['Area Code']<1000)  # only keep the countries. All regions are dropped 
                 & ((df['Item Code']>2900) # only keep category groups.
                 & (df['Unit']!='kg'))     # do not keep kg data (using analysis is on calories per person)
                 | ((df['Unit']=='g/capita/day') & (df['Item Code']==2901)) # or keep Total Protein data.
                )]
    df = df.drop([
                  'Area Code',   # Area Code is a numeric country code not used elsewhere 
                  'Flag',        # Flag is quality of the data
                  'Year Code'    # Duplicate of Year
                 ],axis=1)       # indicator that columnns are to be dropped
    return df

In [47]:
'''
Procedure: loadFiles
Inputs:
    None
Outputs:
    Tuple     two dataframes. One containing income details, the other food & population details.
Purpose:
Load in the two source files. 
Remove unused data to minimize memory usage and improve performance.
'''
def loadFiles():
    foodDataType = { # FAO file structure
                    'Area Code': np.int16,    # index representing the country or region
                    'Area': np.str,           # string of the country
                    'Item Code': np.int16,    # index of the item. Groups > 2900
                    'Item': np.str,           # item or group of food
                    'Element Code': np.int16, # index of element
                    'Element': np.str,        # what is being measured
                    'Year Code': np.int16,    # year
                    'Year': np.int16,         # year
                    'Unit': np.str,           # unit scale - can be used as a label
                    'Value': np.float32,      # 
                    'Flag': np.str            # notes on accuracy
                   }

    # import the World Bank income dataset.
    # note the first four rows are either blank or have date information and are skipped.
    df_GDP = pd.read_csv('Data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_10181232.csv', skiprows=4)
    df_GDP = df_GDP.drop([
                        'Indicator Code',  # data reference - consistent for all records.
                        'Indicator Name',    # three digit country code used in Region Table 
                        '1960'             # 1960 contains very bad data
                        ],axis=1)          # indicator that columnns are to be dropped
    df_GDP.rename({'Value':'Income'})

    # import the Food and Agriculture Organization dataset.
    df_Load = pd.read_csv('Data/FoodBalanceSheets_E_All_Data_(Normalized).csv', 
                          dtype=foodDataType,    # file structure 
                          nrows = 3000,
                          encoding='ISO-8859-1') # file format
    df_Food = removeGranular(df_Load)
    
    df_Regions = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')
        
    return df_Food, df_GDP, df_Regions

In [34]:
'''
Procedure: UpdateValues
Inputs:
    df             pandas dataframe. no specific structure
    updateColumn   name of the column to be checked
    searchFor      string value to search for. This will be dropped from all output unless replaced.
    secondColumn   an optional condition that a second column will also be searched
    searchSecond   what value to search for in the second column 
    replaceWith    Add the specified text where the original matching text existed
    keepBefore     indicator if text before searched value should be kept
    keepAfter      indicator if text after  searched value should be kept
Outputs:
    dataframe
Purpose:
Used to get country names consistent between two datasets.
'''
def UpdateValues(df, updateColumn, searchFor, 
                 secondColumn='', secondSearch='', 
                 replaceWith='', keepBefore=True, keepAfter=True):
    if secondColumn == '':                                          # If search only one parameter
        valueList = df[updateColumn].loc[(                          # create new dataset from updateColumn
            df[updateColumn].str.find(searchFor)>=0)].str.split(    # locate rows where characters (searchFor) exists
            searchFor, expand=True)                                 # split into a columns before and after search string

    else:                                                           # If search two parameters
        valueList = df[updateColumn].loc[((                         # create new dataset from updateColumn
            df[updateColumn].str.find(searchFor)>=0)                # locate rows where characters (searchFor) exists
            & (df[secondColumn] == secondSearch))].str.split(       # and secondColumn is EQUAL to secondSearch
            searchFor, expand=True)                                 # split into a columns before and after search string

    valueList = valueList.replace(np.nan, '', regex=True)           # if any components are blank change to null string
    if valueList.size >0:                                           # if more than one element found then update
        df[updateColumn].update(
            keepBefore * valueList[0]      # KeepBefore (True/False) * (characters before search term) 
            + replaceWith                  # search text removed. add this instead.
            + keepAfter * valueList[1]     # KeepAfter  (True/False) * (characters after search term)
        )

In [5]:
'''
Procedure: fixMappings
Inputs:
    df1            pandas dataframe - FAO
    df2            pandas dataframe - World Bank
Outputs:
    None           original dataframes are modified
Purpose:
Used to get country names consistent between two datasets. All the necessary rules
'''
def fixMappings(df1, df2):
    # first set fix multple inconsistencies
    UpdateValues(df2, 'Country Name', ',', keepAfter=False)
    UpdateValues(df2, 'Country Name', 'St.', replaceWith='Saint')
    UpdateValues(df1, 'Area', ',', keepBefore=False)
    UpdateValues(df1, 'Area', ' \(', keepAfter=False)               # fix Not working
    UpdateValues(df1, 'Area', ' People', keepBefore=False)
    UpdateValues(df1, 'Area', ' Republic of', keepBefore=False)
    UpdateValues(df2, 'Country Name', 'PDR', keepAfter=False)

    # below are country specific updates
    UpdateValues(df2, 'Country Name', 'Czech Republic', replaceWith='Czechoslovakia') 
    UpdateValues(df2, 'Country Name', 'Kyrgyz Republic', replaceWith='Kyrgyzstan')   
    UpdateValues(df2, 'Country Name', 'United States', replaceWith='United States of America')   
    UpdateValues(df1, 'Area', 'd\'', replaceWith='Cote d\'', keepBefore=False)
    UpdateValues(df1, 'Area', 'Viet Nam', replaceWith='Vietnam')
    UpdateValues(df1, 'Element', " \(", keepAfter=False)            # fix Not working
    UpdateValues(df1, 'Item', " -", keepAfter=False)

In [6]:
'''
Procedure: updateMissingData
Inputs:
    df             pandas dataframe - generic
Outputs:
    None           original dataframes are modified
Purpose:
Used to linearly estimate missing data (NaN) in a row
Where a gap exists in the data - the missing data is straight lined between the two points
Where upto three ending points are missing - a linear method is used
Gaps at the beginning of the row (or column) are not populated

*************************************
NOTE - We should set a flag to show the data is estimated. Then if desired we can put colours on the graph.
     - Also useful for 'confidence' in the particular relationship. Very nice for the documentation.
*************************************
'''
def updateMissingData(df, on_axis):
    df = df.interpolate(method='linear', axis=on_axis, limit=10, limit_area='inside')   # fix - change to poly
    df = df.interpolate(method='linear', axis=on_axis, limit=3, limit_area='outside')   # fix - impute
    return df

In [7]:
def regroupRegions (df):
    df['region'].update(df['sub-region'].loc[(
            df['region'].str.find('Americas')>=0)])

    df['region'].update(df['sub-region'].loc[(
            df['region'].str.find('Asia')>=0)])

In [48]:
def __main__():
    food, incomes, regions = loadFiles()
    fixMappings(food, incomes)
    regroupRegions (regions)
            
    # because each row contains headers we only pass the data points.
    incomes[incomes.columns[4:]] = updateMissingData(incomes[incomes.columns[4:]],1)
    
    # undo the pivot table so years are in a single column. This will make graphs easier.
    incomes = pd.melt(incomes, id_vars=['Country Name', 'Country Code'], var_name='Year', value_name='Income')
    incomes['Year'] = pd.to_numeric(incomes['Year'], errors='coerce',downcast= 'integer')
    
    # combine regions and income table = match on 3-digit country code
    df_working = pd.merge (left = incomes, 
                           right = regions[['alpha-3','region']], 
                           how = 'inner', 
                           left_on = 'Country Code',
                           right_on = 'alpha-3')

    df_working = pd.merge (left = df_working,
                           right = food,
                           how = 'inner',
                           left_on = ['Country Name','Year'],
                           right_on = ['Area','Year'])
    df_working.to_csv('working.csv')
    #food.to_csv('food.csv')
    #incomes.to_csv('incomes.csv')
    #return pandas_profiling.ProfileReport(food)
    
    ### combine into a single dataset
    ### df.set_index(['year', 'country', 'field'])


In [49]:
% time __main__() #.to_file("Data_Profile.html")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1113 entries, 53 to 1218
Data columns (total 8 columns):
Area            1113 non-null object
Item Code       1113 non-null int16
Item            1113 non-null object
Element Code    1113 non-null int16
Element         1113 non-null object
Year            1113 non-null int16
Unit            1113 non-null object
Value           1113 non-null float32
dtypes: float32(1), int16(3), object(4)
memory usage: 54.3+ KB
None
Wall time: 1.41 s


In [10]:
'''*************************************
When building graphs put in loops and a delay of a few seconds between each update. This might allow movement in the output.
This way we can build up (keep original if showing 1-5 areas), or hide previous if showing multiple (video)
If doing only a few areas - consider moving labels instead of a legend.

https://scipy-cookbook.readthedocs.io/items/Matplotlib_Animations.html

Can we estimatate and plot the confidence interval?
*************************************'''

'*************************************\nWhen building graphs put in loops and a delay of a few seconds between each update. This might allow movement in the output.\nThis way we can build up (keep original if showing 1-5 areas), or hide previous if showing multiple (video)\nIf doing only a few areas - consider moving labels instead of a legend.\n\nhttps://scipy-cookbook.readthedocs.io/items/Matplotlib_Animations.html\n\nCan we estimatate and plot the confidence interval?\n*************************************'